In [ ]:
import sys
import os
sys.path.append("/home/vtrappler/")
sys.path.append("/home/vtrappler/qgs/")

import warnings
from typing import Callable, Tuple

import matplotlib.pyplot as plt
import numpy as np
import tqdm
from numba.core.errors import NumbaPerformanceWarning

warnings.simplefilter('ignore', category=NumbaPerformanceWarning)
# m = n * (nobs + 1)
from common.numerical_model import NumericalModel
from common.observation_operator import IdentityObservationOperator
from dynamical_systems.quasigeostrophic_numerical_model import create_QG_model

plt.style.use("seaborn")
plt.set_cmap('magma')
from qgs.QG import QGwrapper

# Let us set a rng for reproducibility
rng = np.random.default_rng(seed=93)

qg_model = QGwrapper(wavenumbers=(4, 3), tsteps=0.05, write_steps=1, dt=0.01)
qg_model.configure(tangent_linear=True)
qg_model.burn_model(burn=20_000)

: 

In [ ]:
# plt.subplot(1, 2, 1)
# plt.imshow(np.cov(traj))
# plt.subplot(1, 2, 2)
# plt.imshow(np.linalg.inv(np.cov(traj)))
# np.linalg.slogdet(np.cov(traj))

: 

In [ ]:
import os
if not os.path.exists(f'Binv_{qg_model.x}_{qg_model.y}.npy'):
    time, traj = qg_model.forward(qg_model.initial_conditions, tsteps=10_000, write_steps=10)
    print(traj.shape)
    plt.subplot(1, 2, 1)
    plt.imshow(np.cov(traj))
    plt.subplot(1, 2, 2)
    plt.imshow(np.linalg.inv(np.cov(traj)))
    np.linalg.slogdet(np.cov(traj))
    np.save(f"./Binv_{qg_model.x}_{qg_model.y}.npy", np.linalg.inv(np.cov(traj)))

: 

In [ ]:
# ## Run model for 1 day
# qg_model.change_settings(tsteps=0.5 * qg_model.tstep_1day, write_steps=1, dt=None)
# GN, (time, _, _) = qg_model.GaussNewtonMatrix(qg_model.initial_conditions)
# slogdet = np.linalg.slogdet(GN)
# print(slogdet, np.linalg.cond(GN))
# plt.subplot(1, 2, 1)
# plt.title(np.linalg.slogdet(GN))
# plt.imshow(GN)
# plt.subplot(1, 2, 2)
# plt.title(np.linalg.cond(GN))
# plt.plot(np.linalg.eigvalsh(GN))
# plt.yscale("log")
# plt.tight_layout()

# t, traj, jacobian_ = qg_model.forward_jacobian(x=qg_model.initial_conditions)
# jacobian = jacobian_[:, :, -1]

: 

In [ ]:
tsteps = qg_model.tstep_1day / 1.0
qg_model.change_settings(tsteps=tsteps, write_steps=1, dt=None)
print(f"{(tsteps / qg_model.tstep_1day) * 24:.2f} hours")
obs_operator = IdentityObservationOperator(qg_model.spectral_dim, qg_model.spectral_dim)
qg_model.generate_obs(qg_model.initial_conditions + 0.2, 1)
background = np.zeros(qg_model.spectral_dim), np.load(f"Binv_{qg_model.x}_{qg_model.y}.npy")
# background = None
model, sp_fun, gn_fun = create_QG_model(qg_model, obs_operator=obs_operator, background=background, test=True, gnparams=(50, 20, 0.1))
x0 = np.zeros(qg_model.spectral_dim)

: 

In [ ]:
obs_operator.H.shape

: 

In [ ]:
import scipy
qg_model.generate_obs(qg_model.initial_conditions + np.random.normal(size=qg_model.spectral_dim), 1)
model.set_obs(qg_model.obs)
res = model.GNmethod(np.zeros(qg_model.spectral_dim), n_outer=10, n_inner=2000, verbose=True)
sp_opt = scipy.optimize.minimize(model.cost_function, x0)
sp_x, sp_fun = sp_opt.x, sp_opt.fun
print(f"scipy: {sp_fun}")
print(f"GN: {res[1]}")
plt.plot(res[0], label='GN')
plt.plot(sp_x, label='scipy')
plt.legend()

: 

In [ ]:
import numpy as np
S = np.random.normal(size=(10, 3))
H = np.eye(10) + S @ S.T
Hm1 = np.eye(10) - S @ np.linalg.inv(np.eye(3) + S.T @ S) @ S.T
plt.imshow(H @ Hm1)

: 

In [ ]:
import scipy
for _ in range(5):
    print(scipy.optimize.check_grad(model.cost_function, model.gradient, x0=np.random.normal(size=qg_model.spectral_dim)))

: 

In [ ]:
plt.plot(qg_model.initial_conditions)

: 

In [ ]:
qg_model.generate_obs(qg_model.initial_conditions + 0.2, 0.01)

: 

In [ ]:
eps = 1e-7
x_ = np.random.normal(size=(qg_model.spectral_dim))
cost = qg_model.cost_function(x_)
grad_fd = []
for i in tqdm.trange(qg_model.spectral_dim):
    e = np.zeros(qg_model.spectral_dim)
    e[i] = 1
    grad_fd.append((qg_model.cost_function(x_ + eps * e) - cost) / eps)

: 

In [ ]:
grad = qg_model.gradient(x_)
np.array(grad_fd) - grad

: 

In [ ]:
t, traj, jac = qg_model.forward_jacobian(qg_model.initial_conditions)
jac = jac[..., -1]
GN2 = jac.T @ jac

: 

In [ ]:
GN, _ = qg_model.GaussNewtonMatrix(qg_model.initial_conditions)
np.linalg.slogdet(GN)

: 

In [ ]:
plt.style.available

: 

: 